In [2]:
import pandas as pd
df_p = pd.read_csv("Pandas_data_for_course/Products.csv", sep=';')
df_c = pd.read_csv("Pandas_data_for_course/Customers.csv", sep=';')
df_o = pd.read_csv("Pandas_data_for_course/Orders.csv", sep=';')
df_od = pd.read_csv("Pandas_data_for_course/Order_details.csv", sep=';')
df_e = pd.read_excel("Pandas_data_for_course/Employees.xlsx")

In [4]:
# Convert the columns to datetime
df_o['OrderDate'] = pd.to_datetime(df_o['OrderDate'])
df_o['ShippedDate'] = pd.to_datetime(df_o['ShippedDate'])
df_o['RequiredDate'] = pd.to_datetime(df_o['RequiredDate'])


In [18]:
# приведение все к году, месяцу, дню и неделе
df_o['Year'] = df_o['OrderDate'].dt.year
df_o['Month'] = df_o['OrderDate'].dt.month
df_o['Day'] = df_o['OrderDate'].dt.day
df_o['Quarter'] = df_o['OrderDate'].dt.quarter
df_o['Week_ISO'] = df_o['OrderDate'].dt.isocalendar().week 
df_o.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,Freight,Year,Month,Day,Week_ISO,Quarter
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,32.38,1996,7,4,27,3
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,11.61,1996,7,5,27,3
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,65.83,1996,7,8,28,3
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,41.34,1996,7,8,28,3
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,51.30,1996,7,9,28,3


In [10]:
df_for_pivoting = df_o.groupby(['Year', 'Month']).OrderID.count().reset_index()

In [14]:
df_for_pivoting.pivot_table(index="Year", columns="Month", values="OrderID").reset_index()

Month,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,1996,NaN,NaN,NaN,NaN,NaN,NaN,22.0,25.0,23.0,26.0,25.0,31.0
1,1997,33.0,29.0,30.0,31.0,32.0,30.0,33.0,33.0,37.0,38.0,34.0,48.0
2,1998,55.0,54.0,73.0,74.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_for_pivoting.pivot_table(index="Year", columns="Month", values="OrderID", fill_value=0).reset_index()

Month,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,1996,0.0,0.0,0.0,0.0,0.0,0.0,22.0,25.0,23.0,26.0,25.0,31.0
1,1997,33.0,29.0,30.0,31.0,32.0,30.0,33.0,33.0,37.0,38.0,34.0,48.0
2,1998,55.0,54.0,73.0,74.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_o.pivot_table(index="Year", columns="Month", values="OrderID", fill_value=0, aggfunc="count").reset_index()

Month,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,1996,0,0,0,0,0,0,22,25,23,26,25,31
1,1997,33,29,30,31,32,30,33,33,37,38,34,48
2,1998,55,54,73,74,14,0,0,0,0,0,0,0


In [19]:
df_o.pivot_table(index="Quarter", columns="Year", values="OrderID", fill_value=0, aggfunc="count").reset_index()

Year,Quarter,1996,1997,1998
0,1,0,92,182
1,2,0,93,88
2,3,70,103,0
3,4,82,120,0


## Another way to create DataFrames



In [21]:
df_e['TitleOfCourtesy'].unique()


array(['Ms.', 'Dr.', 'Mrs.', 'Mr.'], dtype=object)

In [22]:
pd.DataFrame({"Title": df_e['Title']})


,Title
0,Sales Representative
1,"Vice President, Sales"
2,Sales Representative
3,Sales Representative
4,Sales Manager
5,Sales Representative
6,Sales Representative
7,Inside Sales Coordinator
8,Sales Representative


In [21]:
df_sex = pd.DataFrame({'TitleOfCourtesy': ['Ms.', 'Dr.', 'Mrs.', 'Mr.'], 'Sex': ['Female', 'Male', 'Female', 'Male']})
df_sex

,TitleOfCourtesy,Sex
0,Ms.,Female
1,Dr.,Male
2,Mrs.,Female
3,Mr.,Male


In [28]:
df_1 = df_sex.merge(df_e).merge(df_o[['EmployeeID', 'OrderID']])
df_sex_orders = df_1[['Sex', 'OrderID']]
sex_counts = df_sex_orders['Sex'].value_counts()
difference = sex_counts['Male'] - sex_counts['Female']
difference

-276

In [37]:
# Шаг 1: Объединение DataFrame'ов
df_1 = df_sex.merge(df_e).merge(df_o[['EmployeeID', 'OrderID']])

# Шаг 2: Создание нового DataFrame с нужными столбцами
df_sex_orders = df_1[['Sex', 'OrderID']]

# Шаг 3: Группировка данных по полу и подсчет количества заказов
sex_counts = df_sex_orders.groupby('Sex').count()
sex_counts




,OrderID
Sex,
Female,553
Male,277


In [34]:
# Шаг 4: Вычисление разницы между количеством заказов, сделанных мужчинами и женщинами
difference = sex_counts.loc['Male', 'OrderID'] - sex_counts.loc['Female', 'OrderID']
difference

-276